In [1]:
from os import listdir, walk
from os.path import isfile, join
import numpy as np
from PIL import Image
import cv2
from IPython.display import display
import math, random
import time, datetime, sys, os, shutil
import operator

import tensorflow as tf
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Input
from tensorflow.keras.utils import Sequence
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.optimizers import *

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split
import pandas as pd

#tf.config.optimizer.set_jit(True)
#tf.debugging.set_log_device_placement(True)
print(tf.config.experimental.list_physical_devices())

AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 32

%run ./variables.ipynb
%run ./utils.ipynb
%run ../utils/data_utils.ipynb

id_map = get_selected_taxons(SELECTED_TAXONS)
id_map_inv = get_selected_taxons(SELECTED_TAXONS, inv=True)
n_classes = len(list(id_map.keys()))

tf.test.is_gpu_available()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'), PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

# Handle data

## Preparing panda arrays

In [2]:
X, y, _ = get_dataset(DATASET_PATH, ids=False)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("Original length:", len(X))
print("Test length:", len(X_train), "- n classes:", len(np.unique(y_train)))
print("Test length:", len(X_test), "- n classes:", len(np.unique(y_test)))
# Balance dataset
train_dict = {}
for file, label in zip(X_train, y_train):
    train_dict.setdefault(label, []).append(file)
max_samples = np.max([len(train_dict[taxon_id]) for taxon_id in train_dict])
X_train = []
y_train = []
for taxon_id in train_dict:
    ratio = np.ceil(max_samples/len(train_dict[taxon_id]))
    tmp = np.repeat(train_dict[taxon_id], ratio)
    np.random.shuffle(tmp)
    train_dict[taxon_id] = tmp[0:max_samples]
    X_train.extend(tmp[0:max_samples])
    y_train.extend([taxon_id]*max_samples)

'171/171'

Original length: 6144
Test length: 4915 - n classes: 171
Test length: 1229 - n classes: 167


In [3]:
data_train = {'png_path':  X_train, 'taxon_id': y_train}
data_test = {'png_path':  X_test, 'taxon_id': y_test}

df_train = pd.DataFrame(data_train, columns = ['png_path', 'taxon_id'])
df_test = pd.DataFrame(data_test, columns = ['png_path', 'taxon_id'])
df_train = df_train.sample(frac=1).reset_index(drop=True)
df_test = df_test.sample(frac=1).reset_index(drop=True)

# Prtining some infos
print(len(df_train), len(df_test))
df_train.head()
df_test.head()

20691 1229


,png_path,taxon_id
0,/mnt/nvme-storage/pfauregi/datasets/condensed/...,NVEN
1,/mnt/nvme-storage/pfauregi/datasets/condensed/...,CMLF
2,/mnt/nvme-storage/pfauregi/datasets/condensed/...,AMID
3,/mnt/nvme-storage/pfauregi/datasets/condensed/...,PSBR
4,/mnt/nvme-storage/pfauregi/datasets/condensed/...,CMEN


## Setting up the flows

In [16]:
datagen = ImageDataGenerator(rescale=1./255.,
                         rotation_range=90, 
                         brightness_range=[0.8,1.2], 
                         horizontal_flip=True, 
                         vertical_flip=True,
                         fill_mode='nearest',
                         width_shift_range=40,
                         height_shift_range=40,
                         zoom_range=0.2,
                         validation_split=0.2,
                         #featurewise_std_normalization=True,
                         data_format="channels_last")

classes_array = id_map
train_generator = datagen.flow_from_dataframe(
        dataframe=df_train,
        x_col='png_path',
        y_col='taxon_id',
        target_size=(256, 256),
        batch_size=32,
        classes=classes_array,
        class_mode='categorical')
val_generator = datagen.flow_from_dataframe(
        dataframe=df_test,
        x_col='png_path',
        y_col='taxon_id',
        target_size=(256, 256),
        batch_size=32,
        classes=classes_array,
        class_mode='categorical')

train_spe = train_generator.samples // BATCH_SIZE
val_spe = val_generator.samples // BATCH_SIZE

Found 20691 validated image filenames belonging to 230 classes.
Found 1229 validated image filenames belonging to 230 classes.


In [19]:
class_indices = train_generator.class_indices
f = open(os.path.join(SAVED_MODELS_ROOT, 'model_id_map.csv'), 'w')
with f:
    writer = csv.writer(f)
    writer.writerow(["taxon", "id"])
    for taxon in class_indices:
        writer.writerow([taxon, class_indices[taxon]])

## Testing

In [ ]:
i = 0
stop = False
for batch in train_generator:
    images = batch[0]
    labels = batch[1]
    for i in range(images.shape[0]):
        print(np.argmax(labels[i]))
        image = (images[i,:,:,:]*255).astype('uint8')
        #print(image)
        display(Image.fromarray(image))
        i+=1
        if i>=1:
            stop = True
            break
    if stop: break;
    #display()

# Model desgin

In [5]:
# fetching base model
#base_model = Xception(include_top=False, weights='imagenet', input_shape=(256, 256, 3), pooling=None)
input_tensor = Input(shape=(256, 256, 3))
base_model = InceptionV3(weights='imagenet', input_tensor=input_tensor, include_top=False)

In [6]:
# setting model for specifiv case
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
out = Dense(230, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=out)

# Training

In [7]:
delete_all_files_in_folder(LOG_DIR)
delete_all_files_in_folder(SAVED_MODELS_ROOT)

## Setting callbacks

In [8]:
# Setting tensorboard
log_dir = LOG_DIR + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [9]:
log_file=os.path.join(SAVED_MODELS_ROOT, "model.log")
csv_logger = CSVLogger(log_file)

## New layers only

In [10]:
n_epochs = 3
last_epoch = get_last_epoch(log_file)

print(n_epochs, "epochs composed of", train_spe, "batches (steps) of", BATCH_SIZE, "images.")

for layer in base_model.layers:
    layer.trainable = False
    
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_generator, 
          epochs=last_epoch+n_epochs, 
          steps_per_epoch=train_spe,
          use_multiprocessing=False, 
          validation_data=val_generator,
          validation_steps=val_spe,
          callbacks=[tensorboard_callback, csv_logger],
          initial_epoch=last_epoch)

3 epochs composed of 646 batches (steps) of 32 images.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 646 steps, validate for 38 steps
Epoch 1/3
646/646 [==============================] - 263s 407ms/step - loss: 2.8096 - accuracy: 0.3524 - val_loss: 4.6205 - val_accuracy: 0.1143
Epoch 2/3
646/646 [==============================] - 259s 401ms/step - loss: 1.4531 - accuracy: 0.5904 - val_loss: 4.7965 - val_accuracy: 0.1645
Epoch 3/3
646/646 [==============================] - 260s 402ms/step - loss: 1.1832 - accuracy: 0.6533 - val_loss: 5.2707 - val_accuracy: 0.1324


## Training 2 last inceptions blocks

In [11]:
n_epochs = 7
last_epoch = get_last_epoch(log_file)

print(n_epochs, "epochs composed of", train_spe, "batches (steps) of", BATCH_SIZE, "images.")

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
    layer.trainable = False
for layer in model.layers[249:]:
    layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit(train_generator, 
          epochs=last_epoch+n_epochs, 
          steps_per_epoch=train_spe,
          use_multiprocessing=False,
          validation_data=val_generator,
          validation_steps=val_spe,
          callbacks=[tensorboard_callback, csv_logger],
          initial_epoch=last_epoch)

7 epochs composed of 646 batches (steps) of 32 images.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 646 steps, validate for 38 steps
Epoch 3/9
646/646 [==============================] - 264s 408ms/step - loss: 0.9986 - accuracy: 0.6970 - val_loss: 5.1816 - val_accuracy: 0.2336
Epoch 4/9
646/646 [==============================] - 260s 403ms/step - loss: 0.5358 - accuracy: 0.8258 - val_loss: 4.1440 - val_accuracy: 0.2763
Epoch 5/9
646/646 [==============================] - 262s 406ms/step - loss: 0.4280 - accuracy: 0.8602 - val_loss: 4.4547 - val_accuracy: 0.3207
Epoch 6/9
646/646 [==============================] - 260s 403ms/step - loss: 0.3535 - accuracy: 0.8815 - val_loss: 3.6655 - val_accuracy: 0.3586
Epoch 7/9
646/646 [==============================] - 260s 402ms/step - loss: 0.3129 - accuracy: 0.8938 - val_loss: 4.5725 - val_accuracy: 0.2911
Epoch 8/9
646/646 [==============================] - 261s 403ms/step - loss: 0.2832 - accuracy: 0.9057 - val_loss: 4.6052 - val

## Training full model

## Saving model

In [12]:
n_epochs = 24
last_epoch = get_last_epoch(log_file)

print(n_epochs, "epochs composed of", train_spe, "batches (steps) of", BATCH_SIZE, "images.")

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers:
    layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit(train_generator, 
          epochs=last_epoch+n_epochs, 
          steps_per_epoch=train_spe,
          use_multiprocessing=False, 
          validation_data=val_generator,
          validation_steps=val_spe,
          callbacks=[tensorboard_callback, csv_logger],
          initial_epoch=last_epoch)

24 epochs composed of 646 batches (steps) of 32 images.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 646 steps, validate for 38 steps
Epoch 9/32
646/646 [==============================] - 265s 410ms/step - loss: 0.9534 - accuracy: 0.7148 - val_loss: 1.2955 - val_accuracy: 0.6456
Epoch 10/32
646/646 [==============================] - 260s 402ms/step - loss: 0.4702 - accuracy: 0.8437 - val_loss: 1.9315 - val_accuracy: 0.5946
Epoch 11/32
646/646 [==============================] - 261s 403ms/step - loss: 0.3496 - accuracy: 0.8834 - val_loss: 1.7296 - val_accuracy: 0.5929
Epoch 12/32
646/646 [==============================] - 264s 409ms/step - loss: 0.2975 - accuracy: 0.9027 - val_loss: 1.4118 - val_accuracy: 0.6735
Epoch 13/32
646/646 [==============================] - 260s 402ms/step - loss: 0.2660 - accuracy: 0.9103 - val_loss: 1.0759 - val_accuracy: 0.7220
Epoch 14/32
646/646 [==============================] - 259s 401ms/step - loss: 0.2420 - accuracy: 0.9183 - val_loss: 

In [13]:
save_model(model, SAVED_MODELS_ROOT)

Saved model to ./saved_models/model.json
Saved weights to ./saved_models/model.h5
